In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ==========================
# Load and prepare data
# ==========================
# Assuming file already read and cleaned into dfc (with columns: date, browser, visitors, successful_order_landing_page, aov, #items)

# Example: if starting fresh, load with
# df = pd.read_excel("Problem Statement.xlsx", header=[0,1,2])
# (data cleaning steps omitted here since we already parsed earlier)

# ==========================
# Daily summary
# ==========================
daily = (
    dfc.groupby(["date","browser"], as_index=False)
       .agg(visitors=("visitors","sum"),
            orders=("successful_order_landing_page","sum"),
            aov=("aov","mean"),
            items=("#items","sum"))
)
daily["conversion"] = daily["orders"] / daily["visitors"]
daily["revenue"] = daily["orders"] * daily["aov"]

# ==========================
# Graphs: Revenue, Conversion, AOV
# ==========================
plt.figure(figsize=(10,5))
for ch, sub in daily.groupby("browser"):
    plt.plot(sub["date"], sub["revenue"], label=f"{ch} revenue")
plt.title("Revenue over time by channel")
plt.xlabel("Date")
plt.ylabel("Revenue")
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,5))
for ch, sub in daily.groupby("browser"):
    plt.plot(sub["date"], sub["conversion"], label=f"{ch} conversion")
plt.title("Conversion rate over time by channel")
plt.xlabel("Date")
plt.ylabel("Conversion rate")
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,5))
for ch, sub in daily.groupby("browser"):
    plt.plot(sub["date"], sub["aov"], label=f"{ch} AOV")
plt.title("Average Order Value over time")
plt.xlabel("Date")
plt.ylabel("AOV")
plt.legend()
plt.tight_layout()
plt.show()

# ==========================
# Funnel checkout → order success rate
# ==========================
dfc["checkout_to_order"] = dfc["successful_order_landing_page"] / dfc["checkout"]
conv_step = (
    dfc.groupby(["date","browser"], as_index=False)["checkout_to_order"].mean()
)

plt.figure(figsize=(10,5))
for ch, sub in conv_step.groupby("browser"):
    plt.plot(sub["date"], sub["checkout_to_order"], label=ch)
plt.title("Checkout → Order success rate")
plt.xlabel("Date")
plt.ylabel("Success rate")
plt.legend()
plt.tight_layout()
plt.show()

# ==========================
# Items per order
# ==========================
by_browser_extra = dfc.groupby(["date","browser"], as_index=False).agg(
    items=("#items","sum"),
    orders=("successful_order_landing_page","sum"),
    aov=("aov","mean")
)
by_browser_extra["items_per_order"] = by_browser_extra["items"] / by_browser_extra["orders"]

plt.figure(figsize=(10,5))
for ch, sub in by_browser_extra.groupby("browser"):
    plt.plot(sub["date"], sub["items_per_order"], label=ch)
plt.title("Items per order by channel")
plt.xlabel("Date")
plt.ylabel("Items per order")
plt.legend()
plt.tight_layout()
plt.show()

# ==========================
# Extension 1: Conversion by weekday
# ==========================
dfc["weekday"] = dfc["date"].dt.day_name()
weekday_conv = (
    dfc.groupby(["weekday","browser"], as_index=False)
       .agg(visitors=("visitors","sum"), orders=("successful_order_landing_page","sum"))
)
weekday_conv["conversion"] = weekday_conv["orders"] / weekday_conv["visitors"]

cats = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
weekday_conv["weekday"] = pd.Categorical(weekday_conv["weekday"], categories=cats, ordered=True)
weekday_conv = weekday_conv.sort_values("weekday")

plt.figure(figsize=(8,5))
for ch, sub in weekday_conv.groupby("browser"):
    plt.plot(sub["weekday"], sub["conversion"], marker="o", label=ch)
plt.title("Conversion rate by weekday")
plt.xlabel("Weekday")
plt.ylabel("Conversion rate")
plt.legend()
plt.tight_layout()
plt.show()

# ==========================
# Extension 2: Anomaly detection (Z-score)
# ==========================
daily["conv_zscore"] = (
    daily.groupby("browser")["conversion"]
         .transform(lambda x: (x - x.mean())/x.std())
)
daily["anomaly_flag"] = daily["conv_zscore"] < -2

plt.figure(figsize=(10,5))
for ch, sub in daily.groupby("browser"):
    plt.plot(sub["date"], sub["conversion"], label=ch)
    plt.scatter(sub.loc[sub["anomaly_flag"], "date"],
                sub.loc[sub["anomaly_flag"], "conversion"],
                color="red", marker="x", s=80, label=f"{ch} anomaly")
plt.title("Conversion with anomaly detection (z-score)")
plt.xlabel("Date")
plt.ylabel("Conversion rate")
plt.legend()
plt.tight_layout()
plt.show()

# ==========================
# (Optional) Extension 2b: Prophet-based anomaly detection
# ==========================
# from prophet import Prophet
# app_daily = daily[daily["browser"]=="App"][["date","conversion"]].rename(columns={"date":"ds","conversion":"y"})
# m = Prophet(interval_width=0.95)
# m.fit(app_daily)
# future = m.make_future_dataframe(periods=0)
# forecast = m.predict(future)
# merged = app_daily.merge(forecast[["ds","yhat","yhat_lower","yhat_upper"]], on="ds")
# merged["anomaly"] = merged["y"] < merged["yhat_lower"]
#
# plt.figure(figsize=(10,5))
# plt.plot(merged["ds"], merged["y"], label="Actual conversion")
# plt.plot(merged["ds"], merged["yhat"], label="Forecast")
# plt.fill_between(merged["ds"], merged["yhat_lower"], merged["yhat_upper"], alpha=0.2)
# plt.scatter(merged.loc[merged["anomaly"], "ds"], merged.loc[merged["anomaly"], "y"], color="red", marker="x", s=80, label="Anomaly")
# plt.title("App Conversion with Prophet anomaly detection")
# plt.legend()
# plt.show()
